In [1]:
import torch
import dgl
import numpy as np
import pandas as pd

d:\Miniconda3\envs\8007project\lib\site-packages\torchdata\datapipes\__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()


In [ ]:
edge_file_path = '../cleaned_data/graph_data/edge_ID.txt'
node_type_file_path = '../cleaned_data/graph_data/node_type_ID.txt'

## Process Nodes

In [5]:
nodes = pd.read_csv(node_type_file_path, sep='\t', header=None)
nodes.columns = ['ID', 'Type']
nodes.head()

,ID,Type
0,OhECKhQEexFypOMY6kypRw,U
1,4hBhtCSgoxkrFgHa4YAD-w,U
2,UsBxLh14sUpO8SdeqIiGOA,U
3,C_2mNjl-doRVvsL03_T57Q,U
4,RiQWbgwR-asiU_mA7s8gpg,U


In [17]:
user_node_set = nodes[nodes['Type']=='U']['ID'].to_list()
poi_node_set = nodes[nodes['Type']=='P']['ID'].to_list()

In [18]:
user_node_set_map = { node_id: i  for i, node_id in enumerate(user_node_set) }
poi_node_set_map  = { node_id: i  for i, node_id in enumerate(poi_node_set) }

In [ ]:
'''
在DGL的图里，所有的边都是有方向的， 即边 (u,v)  表示它是从节点 u  指向节点 v  的。
所以最后是有四种边
'''
count=0
user_user_edge = []
user_user_edge_weight=[]

poi_poi_edge = []
poi_poi_edge_weight=[]

user_poi_edge = []  
user_poi_edge_weight=[]

poi_user_edge = [] 
poi_user_edge_weight=[]

with open(edge_file_path) as f:
        lines = f.readlines()
        for line in lines:
            line_content = line.strip().split("\t")
            source_node = line_content[0]
            target_node = line_content[1]
            edge_weight = eval(line_content[2])
            if node_type_index[source_node] == 'P' and node_type_index[target_node] == 'P': #POI之间的边
                poi_poi_edge.append(( poi_node_set_map[ source_node]  , poi_node_set_map[ target_node]        )  )
                poi_poi_edge.append(( poi_node_set_map[ target_node]  , poi_node_set_map[ source_node]        )  )  # 此行把 POI边变成双向的
                
                poi_poi_edge_weight.append( edge_weight)
                poi_poi_edge_weight.append( edge_weight) 
                
            elif node_type_index[source_node] == 'U' and  node_type_index[target_node] == 'U':   #user之间的边
                user_user_edge.append(( user_node_set_map[ source_node]  , user_node_set_map[ target_node]        )  )
                user_user_edge.append(( user_node_set_map[ target_node]  , user_node_set_map[ source_node]        )  ) # 此行把 User边变成双向的
                
                user_user_edge_weight.append( edge_weight)
                user_user_edge_weight.append( edge_weight)  #边重复了两次，权值自然要重复两次
            else:
                if node_type_index[source_node] == 'U':
                    user_poi_edge.append(( user_node_set_map[ source_node]  , poi_node_set_map[ target_node])  )
                    edge_weight=edge_weight*(dic[(source_node,target_node)]-1)*0.25
                        
                    user_poi_edge_weight.append( edge_weight)
                    
                    poi_user_edge.append(( poi_node_set_map[ target_node]  , user_node_set_map[ source_node]    )  )
                    poi_user_edge_weight.append( edge_weight)
                else:  #source_node P
                    poi_user_edge.append(( poi_node_set_map[ source_node]  , user_node_set_map[ target_node]    )  )
                    edge_weight=edge_weight*(dic[(source_node,target_node)]-1)*0.25
                    poi_user_edge_weight.append( edge_weight)
                    
                    user_poi_edge.append(( user_node_set_map[ target_node]  , poi_node_set_map[ source_node]        )  )
                    user_poi_edge_weight.append( edge_weight)

In [15]:
user_user_edge_array = np.array(user_user_edge)
user_user_edge_weight_array=np.array(user_user_edge_weight)#

poi_poi_edge_array = np.array(poi_poi_edge)
poi_poi_edge_weight_array =np.array(poi_poi_edge_weight)#

user_poi_edge_array  = np.array(user_poi_edge)#
user_poi_edge_weight_array  =np.array(user_poi_edge_weight)#

poi_user_edge_array  = np.array(poi_user_edge)#
poi_user_edge_weight_array  = np.array(poi_user_edge_weight)#

In [16]:
graph_data = {
    ('user', 'user_user', 'user'): (torch.tensor(user_user_edge_array[:,0]), torch.tensor(user_user_edge_array[:,1])),
    ('user', 'user_poi', 'poi'): (torch.tensor(user_poi_edge_array[:,0]), torch.tensor(user_poi_edge_array[:,1])),
    ('poi', 'poi_user', 'user'): (torch.tensor(poi_user_edge_array[:,0]), torch.tensor(poi_user_edge_array[:,1])),
    ('poi', 'poi_poi', 'poi'): (torch.tensor(poi_poi_edge_array[:,0]), torch.tensor(poi_poi_edge_array[:,1]))
}
g = dgl.heterograph(graph_data)  #构建图

In [17]:
print(g.ntypes)
print(g.metagraph().edges())
print("user_num_nodes",g.num_nodes('user'))
print("poi_num_nodes",g.num_nodes('poi'))
print(g.etypes)
print(g.canonical_etypes)

['poi', 'user']
[('poi', 'poi'), ('poi', 'user'), ('user', 'poi'), ('user', 'user')]
user_num_nodes 6379
poi_num_nodes 1434
['poi_poi', 'poi_user', 'user_poi', 'user_user']
[('poi', 'poi_poi', 'poi'), ('poi', 'poi_user', 'user'), ('user', 'user_poi', 'poi'), ('user', 'user_user', 'user')]


In [18]:
#添加边特征
g.edges['user_user'].data['w'] = torch.tensor ( user_user_edge_weight_array  , dtype=torch.float32)   
g.edges['user_poi'].data['w'] = torch.tensor( user_poi_edge_weight_array  , dtype=torch.float32)   
g.edges['poi_user'].data['w'] = torch.tensor( poi_user_edge_weight_array  , dtype=torch.float32) 
g.edges['poi_poi'].data['w'] = torch.tensor( poi_poi_edge_weight_array  , dtype=torch.float32)  

In [19]:
g

Graph(num_nodes={'poi': 1434, 'user': 6379},
      num_edges={('poi', 'poi_poi', 'poi'): 320, ('poi', 'poi_user', 'user'): 108312, ('user', 'user_poi', 'poi'): 108312, ('user', 'user_user', 'user'): 82808},
      metagraph=[('poi', 'poi', 'poi_poi'), ('poi', 'user', 'poi_user'), ('user', 'poi', 'user_poi'), ('user', 'user', 'user_user')])

In [ ]:
poi_fe=np.load('../cleaned_data/graph_data/poi_fe.npy')
user_fe=np.load('../cleaned_data/graph_data/user_fe.npy')

In [ ]:
g.nodes['user'].data['fe']=torch.tensor(user_fe, dtype=torch.float32)
g.nodes['poi'].data['fe']=torch.tensor(poi_fe, dtype=torch.float32)   

In [22]:
from dgl import save_graphs, load_graphs

In [23]:
torch.save(g, 'test')

In [24]:
m= torch.load('test')

In [25]:
m

Graph(num_nodes={'poi': 1434, 'user': 6379},
      num_edges={('poi', 'poi_poi', 'poi'): 320, ('poi', 'poi_user', 'user'): 108312, ('user', 'user_poi', 'poi'): 108312, ('user', 'user_user', 'user'): 82808},
      metagraph=[('poi', 'poi', 'poi_poi'), ('poi', 'user', 'poi_user'), ('user', 'poi', 'user_poi'), ('user', 'user', 'user_user')])